<a href="https://colab.research.google.com/github/123hawaa/Computer_vision_project/blob/main/rock_paper_scissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q tensorflow tensorflow-datasets

# Import Necessary Libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
tfds.disable_progress_bar()

# Find Datasets

In [3]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asqa',
 'asset',
 'assin2',
 'asu_table_top_converted_externally_to_rlds',
 'austin_buds_dataset_converted_externally_to_rlds',
 'austin_sailor_dataset_converted_externally_to_rlds',
 'austin_sirius_dataset_converted_externally_to_rlds',
 'bair_robot_pushing_small',
 'bc_z',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'berkeley_autolab_ur5',
 'berkeley_cable_routing',
 'berkeley_fanuc_manipulation',
 'berkeley_gnm_cory_hall',
 'berkeley_gnm_recon',
 'berkeley_gnm_sac_son',
 'berkeley_mvp_converted_externally_to_rlds',
 'berkeley_rpt_converted_externally_to_rlds',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'ble_wind_field',
 'blimp',
 'booksum',
 'bool_q',
 'bot_adversarial_dialogue',
 'bridge',
 'bucc',
 'c4',
 'c4_wsrs',
 'caltech101',
 'caltech_birds2010',
 'caltech_b

In [4]:
builder = tfds.builder('rock_paper_scissors')
info = builder.info
info

tfds.core.DatasetInfo(
    name='rock_paper_scissors',
    full_name='rock_paper_scissors/3.0.0',
    description="""
    Images of hands playing rock, paper, scissor game.
    """,
    homepage='http://laurencemoroney.com/rock-paper-scissors-dataset',
    data_dir=PosixGPath('/tmp/tmpolh9rrhutfds'),
    file_format=tfrecord,
    download_size=219.53 MiB,
    dataset_size=Unknown size,
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=3),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=372, num_shards=1>,
        'train': <SplitInfo num_examples=2520, num_shards=2>,
    },
    citation="""@ONLINE {rps,
    author = "Laurence Moroney",
    title = "Rock, Paper, Scissors Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/rock-paper-scissors-dataset"
    }""",
)

# Prepare rock_paper_scissors Data


In [5]:
ds_train = tfds.load(name='rock_paper_scissors',split='train')
ds_test = tfds.load(name='rock_paper_scissors',split='test')

Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


In [6]:
#ig = tfds.show_examples(info,ds_train)

# Data Pre
Convert the data into numpy format

In [7]:
train_img = [example['image'] for example in ds_train]

In [8]:
type(train_img[0])

tensorflow.python.framework.ops.EagerTensor

In [9]:
train_img=np.array([example['image'].numpy()for example in ds_train])

In [10]:
type(train_img[0])

numpy.ndarray

In [11]:
train_img.shape

(2520, 300, 300, 3)

In [12]:
train_img=np.array([example['image'].numpy()[:,:,0]for example in ds_train])

In [13]:
train_img.shape

(2520, 300, 300)

In [14]:
train_label=np.array([example['label'].numpy()for example in ds_train])
test_img=np.array([example['image'].numpy()[:,:,0]for example in ds_test])
test_label=np.array([example['label'].numpy()for example in ds_test])

In [15]:
test_img.shape

(372, 300, 300)

In [16]:
#when we use CNN we must say hey this greyscale
train_img=train_img.reshape (2520,300,300,1)
test_img = test_img.reshape (372, 300, 300,1)

In [17]:
train_img.dtype

dtype('uint8')

In [18]:
#when we work with CNN or image classifiction we work with float value specially between 0-1
train_img = train_img.astype('float32')
test_img = test_img.astype('float32')
train_img /=255
test_img/=255

In [19]:
train_img.dtype

dtype('float32')

In [20]:
train_img[0]

array([[[0.99607843],
        [0.99215686],
        [0.99607843],
        ...,
        [0.9843137 ],
        [0.98039216],
        [0.98039216]],

       [[0.99607843],
        [0.99607843],
        [0.99215686],
        ...,
        [0.98039216],
        [0.9843137 ],
        [0.9764706 ]],

       [[0.99607843],
        [0.99607843],
        [0.99607843],
        ...,
        [0.9843137 ],
        [0.98039216],
        [0.9882353 ]],

       ...,

       [[0.9882353 ],
        [0.9843137 ],
        [0.9882353 ],
        ...,
        [0.96862745],
        [0.9764706 ],
        [0.972549  ]],

       [[0.99215686],
        [0.99215686],
        [0.9843137 ],
        ...,
        [0.972549  ],
        [0.972549  ],
        [0.972549  ]],

       [[0.9882353 ],
        [0.99215686],
        [0.9882353 ],
        ...,
        [0.972549  ],
        [0.96862745],
        [0.98039216]]], dtype=float32)

#Train a Network(basic approach)

In [21]:
model=keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(512,activation='relu'),
    keras.layers.Dense(256,activation='relu'),
    keras.layers.Dense(3,activation='softmax')]

)


In [22]:
model.compile(
    optimizer='adam',
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

32 samples from the training dataset will be used to estimate the error gradient before the model weights are updated.

In [23]:
model.fit(train_img,train_label,epochs=5,batch_size=32)

Epoch 1/5
79/79 [==============================] - 4s 21ms/step - loss: 30.1646 - accuracy: 0.3782
Epoch 2/5
79/79 [==============================] - 2s 22ms/step - loss: 2.0321 - accuracy: 0.6560
Epoch 3/5
79/79 [==============================] - 2s 19ms/step - loss: 1.4352 - accuracy: 0.6802
Epoch 4/5
79/79 [==============================] - 2s 19ms/step - loss: 0.4632 - accuracy: 0.8357
Epoch 5/5
79/79 [==============================] - 2s 19ms/step - loss: 0.4434 - accuracy: 0.8353


In [24]:
model.evaluate(test_img,test_label)

12/12 [==============================] - 0s 8ms/step - loss: 1.8676 - accuracy: 0.5591


[1.8676400184631348, 0.5591397881507874]

#Train Network(Convolution approach)

In [25]:
model= keras.Sequential([
    keras.layers.AveragePooling2D(6,3,input_shape=(300,300,1)),
    keras.layers.Conv2D(64,3,activation='relu'),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Conv2D(32,3,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Flatten(),
    keras.layers.Dense(3,activation='softmax')
])


In [26]:
model.compile(
    optimizer='adam',
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [27]:
model.fit(train_img,train_label,epochs=5,batch_size=32)

Epoch 1/5
79/79 [==============================] - 6s 22ms/step - loss: 0.9566 - accuracy: 0.5758
Epoch 2/5
79/79 [==============================] - 1s 18ms/step - loss: 0.3675 - accuracy: 0.8948
Epoch 3/5
79/79 [==============================] - 1s 18ms/step - loss: 0.1466 - accuracy: 0.9722
Epoch 4/5
79/79 [==============================] - 1s 18ms/step - loss: 0.0815 - accuracy: 0.9837
Epoch 5/5
79/79 [==============================] - 1s 18ms/step - loss: 0.0498 - accuracy: 0.9881


In [28]:
model.evaluate(test_img,test_label)

12/12 [==============================] - 0s 26ms/step - loss: 0.6155 - accuracy: 0.7608


[0.6155309081077576, 0.7607526779174805]

#Hyperparametre Tuning


In [29]:
pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 4.1 MB/s eta 0:00:00


In [30]:
import keras_tuner

In [42]:
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.AveragePooling2D(6,3,input_shape=(300,300,1)))
  for i in range(hp.Int("cov layers",min_value=0,max_value=3)):
     model.add(keras.layers.Conv2D(hp.Choice("layer{i}_filters",[16,32,64]),3,activation='relu'))
  model.add(keras.layers.MaxPool2D(2,2))

  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(hp.Choice('Dence layer',[64,128,256,512,1024]),activation='relu'))
  model.add(keras.layers.Dense(3,activation='softmax'))
  model.compile(
    optimizer='adam',
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])
  return model
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_accuracy',
    directory='./new_dic',
    max_trials=5)
tuner.search(train_img,train_label,validation_data=(test_img,test_label),epochs=10,batch_size=32)

Trial 5 Complete [00h 00m 18s]
val_accuracy: 0.7553763389587402

Best val_accuracy So Far: 0.7553763389587402
Total elapsed time: 00h 01m 17s


In [43]:
best_model = tuner.get_best_models()[0]

In [44]:
print(best_model)

In [33]:
best_model.summary

<bound method Model.summary of <keras.src.engine.sequential.Sequential object at 0x7bd154b48970>>

#save and load our models